In [1]:
import json
from pprint import pprint 

import openeo
from openeo.processes import ProcessBuilder, lt, if_
from pg_to_evalscript.conversion import convert_from_process_graph

In [2]:
c = openeo.connect("https://openeo.vito.be/openeo/1.0")

In [3]:
test_polygon = {
    "type": "Polygon",
    "coordinates": [
        [
            [12.481616, 42.016518],
            [12.437678, 41.995082],
            [12.542031, 41.942992],
            [12.632654, 41.971595],
            [12.620297, 42.002228],
            [12.481616, 42.016518],
        ]
    ],
}

In [4]:
def mask(value: ProcessBuilder):
    red = value.array_element(0)
    nir = value.array_element(1)
    clm = value.array_element(2)

    ndvi = (nir - red) / (nir + red)

    return if_(lt(clm, 40), ndvi)

sen2cor = c.load_collection(
    "SENTINEL2_L2A_SENTINELHUB", bands=["B04", "B08", "CLP"], temporal_extent=["2020-01-01", "2021-01-01"]
)
sen2cor_masked = sen2cor.reduce_dimension(dimension="bands", reducer=mask)
sen2cor_masked = sen2cor_masked.aggregate_spatial(test_polygon, reducer="median")
process_graph = json.loads(sen2cor_masked.to_json())["process_graph"]
pprint(process_graph)

{'aggregatespatial1': {'arguments': {'data': {'from_node': 'reducedimension1'},
                                     'geometries': {'coordinates': [[[12.481616,
                                                                      42.016518],
                                                                     [12.437678,
                                                                      41.995082],
                                                                     [12.542031,
                                                                      41.942992],
                                                                     [12.632654,
                                                                      41.971595],
                                                                     [12.620297,
                                                                      42.002228],
                                                                     [12.481616,
                        

In [5]:
evalscripts = convert_from_process_graph(process_graph)
print(evalscripts)

All nodes valid? True
[]
All nodes valid? True
[]
[{'evalscript': <pg_to_evalscript.evalscript.Evalscript object at 0x0000029A85003CA0>, 'invalid_node_id': 'aggregatespatial1'}]


In [6]:
evalscripts[0]["evalscript"].write()

[{'name': 't', 'size': None, 'original_temporal': True}]
1 []


'\n//VERSION=3\nfunction setup() {\n  return {\n    input: [\'B04\',\'B08\',\'CLP\'],\n    output: { bands: 1, sampleType: "AUTO" },\n    mosaicking: "ORBIT"\n  };\n}\n\nfunction updateOutput(outputs, collection) {\n    Object.values(outputs).forEach((output) => {\n        output.bands = 1 + 0 * collection.scenes.length;\n    });\n}\nclass DataCube {\r\n    constructor(data, bands_dimension_name, temporal_dimension_name, fromSamples) {\r\n        this.TEMPORAL = "temporal"\r\n        this.BANDS = "bands"\r\n        this.OTHER = "other"\r\n        this.bands_dimension_name = bands_dimension_name;\r\n        this.temporal_dimension_name = temporal_dimension_name;\r\n        this.dimensions = [{\r\n            name: this.temporal_dimension_name,\r\n            labels: [],\r\n            type: this.TEMPORAL\r\n        }, {\r\n            name: this.bands_dimension_name,\r\n            labels: [],\r\n            type: this.BANDS\r\n        }]\r\n        if (fromSamples) {\r\n            thi